In [1]:
basedir = './imgs/'

In [2]:
import os
import numpy as np
import pandas as pd

from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.applications import *
from keras.preprocessing.image import *

np.random.seed(2017)

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


ModuleNotFoundError: No module named 'keras'

In [3]:
def img_gen(group, gen, basedir, model_image_size, batch_size):
    train_generator = gen.flow_from_directory(os.path.join(basedir, 'train'+str(group)),  model_image_size, \
                                                shuffle=True, batch_size=batch_size, class_mode="categorical")

    valid_generator = gen.flow_from_directory(os.path.join(basedir, 'valid'+str(group)),  model_image_size, \
                                          shuffle=True, batch_size=batch_size, class_mode="categorical")
    
    steps_train_sample = int(train_generator.samples)//batch_size + 1
    steps_valid_sample = int(valid_generator.samples)//batch_size + 1
    return train_generator, valid_generator, steps_train_sample, steps_valid_sample 

In [4]:
def build_model(model_image_size):
    inputs = Input((*model_image_size, 3))
    base_model = VGG16(input_tensor=inputs, weights='imagenet', include_top=False)
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.5)(x)
    x = Dense(10, activation='softmax')(x)
    model = Model(inputs, x)
    for i in range(len(base_model.layers)):
        model.layers[i].trainable = False
    return model

In [5]:
def train_model(model, train_generator, valid_generator, steps_train_sample, steps_valid_sample):
    model.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit_generator(train_generator, steps_per_epoch=steps_train_sample, epochs=1, \
                    validation_data=valid_generator, validation_steps=steps_valid_sample)

In [6]:
def predict_model(model, test_generator, steps_test_sample):
    y_pred = model.predict_generator(test_generator, steps=test_generator.samples//batch_size+1, verbose=1)
    return y_pred

In [7]:
def submission(result, fname):
    results = np.array([])
    for i, imgname in enumerate(test_generator.filenames):
        name = imgname[imgname.rfind('/')+1:]
        results.append([name, *result[i]])
    data = {'img': results[:,0]}
    for i in range(10):
        data['c%d'%i] = results[:,i+1]
    df = pd.DataFrame(data, columns=['img'] + ['c%d'%i for i in range(10)])
    df = df.sort_values(by='img')
    df.to_csv(fname, index=None)    

In [8]:
from keras.applications.vgg16 import preprocess_input

In [20]:
gen = ImageDataGenerator(preprocessing_function=preprocess_input)
train_generator = gen.flow_from_directory(os.path.join(basedir, 'test'), target_size=(224,224), batch_size=16, shuffle=False, class_mode='categorical')

Found 0 images belonging to 0 classes.


In [21]:
model_image_size = (224, 224)
batch_size = 128

gen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = gen.flow_from_directory(os.path.join(basedir, 'test'), target_size=model_image_size, shuffle=False, batch_size=batch_size, class_mode=None)
steps_test_sample = test_generator.samples//batch_size + 1

result = np.array([0.]*79726)
for i in range(1,2):
    train_generator, valid_generator, steps_train_sample, steps_valid_sample = img_gen(i, gen, basedir, model_image_size, batch_size)
    model = build_model(model_image_size)
    train_model(model, train_generator, valid_generator, steps_train_sample, steps_valid_sample)
#    result += predict_model(model, test_generator, steps_test_sample)
#result /= 5

#submission(result, 'VGG16.csv')

Found 0 images belonging to 0 classes.
Found 22424 images belonging to 10 classes.
Found 0 images belonging to 10 classes.


ImportError: `load_weights` requires h5py.